In [ ]:
# Import the libraries we need for this lab

# Using the following line code to install the torchvision library
# !conda install -y torchvision
# Import PyTorch Modules will be used in the lab

import torch 
from torch.autograd import backward
from torch.utils.data import Dataset, DataLoader
import pandas
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pylab as plt
import numpy as np
import torchvision.models as models

In [ ]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import xgboost as xgb
import matplotlib
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline 

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)

import lightgbm as lgb
from sklearn.decomposition import NMF
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import product


def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tungbinhthuong","key":"fd622c39c04da294c5e392c7f76ec1a7"}'}

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle competitions download -c Kannada-MNIST
!ls

  0% 0.00/1.90M [00:00<?, ?B/s]
100% 1.90M/1.90M [00:00<00:00, 63.0MB/s]
  0% 0.00/818k [00:00<?, ?B/s]
100% 818k/818k [00:00<00:00, 110MB/s]
  0% 0.00/33.1k [00:00<?, ?B/s]
100% 33.1k/33.1k [00:00<00:00, 33.5MB/s]
 52% 5.00M/9.71M [00:00<00:00, 20.5MB/s]
100% 9.71M/9.71M [00:00<00:00, 32.3MB/s]
Dig-MNIST.csv.zip  sample_data		  test.csv.zip
kaggle.json	   sample_submission.csv  train.csv.zip


In [ ]:
!unzip Dig-MNIST.csv.zip
!unzip test.csv.zip
!unzip train.csv.zip

Archive:  Dig-MNIST.csv.zip
  inflating: Dig-MNIST.csv           
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
valid = pd.read_csv('/content/Dig-MNIST.csv')

In [ ]:
len(train)

60000

In [ ]:
def read_inp(inp):
    target = inp['label']
    img = inp.drop(['label'], axis = 1)
    img = img.to_numpy()
    img = img.reshape((len(inp),1,28,28)).astype(np.uint8)
    return torch.tensor(img), target 

In [ ]:
#plot_data
def show_data(img, label, index):
  plt.imshow(img[index], cmap='gray')
  plt.title('y = ' + str(label[index]))

In [ ]:
train_img, train_label = read_inp(train)
valid_img, valid_label = read_inp(valid)
train_img = train_img/255 #normalization
valid_img = valid_img/255

In [ ]:
train_img.shape

torch.Size([60000, 1, 28, 28])

In [ ]:
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck

class MNISTResNet(ResNet):
    def __init__(self):
        super(MNISTResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10) # Based on ResNet18
        # super(MNISTResNet, self).__init__(BasicBlock, [3, 4, 6, 3], num_classes=10) # Based on ResNet34
        # super(MNISTResNet, self).__init__(Bottleneck, [3, 4, 6, 3], num_classes=10) # Based on ResNet50
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=1, padding=3,bias=False)

model = MNISTResNet()
print(model)

MNISTResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=Tru

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
lr = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=lr) 

In [ ]:
# Create Dateaset Class

class Dataset(Dataset):
    
    # Constructor
    def __init__(self, img, label, transform=None):
      self.img = img
      self.label = label
      self.transform = transform

    
    # Get Length
    def __len__(self):
      return self.img.shape[0]
    
    # Getter
    def __getitem__(self, idx):
      image = self.img[idx]
      y = self.label[idx]
      if(self.transform != None):
        image = self.transform
      return image, y

In [ ]:
train_data = Dataset(train_img, train_label)
valid_data = Dataset(valid_img, valid_label)

In [ ]:
trainloader = DataLoader(dataset = train_data, batch_size = 64)
validloader = DataLoader(dataset = valid_data, batch_size = 64)

In [ ]:
# Step 4: Train the model

N_EPOCHS = 20
loss_list = []
accuracy_list = []
correct = 0
n_test = len(valid)
for e in range(N_EPOCHS):
  print("epoch: "+str(e))
  model.train()
  for idx, (image, y) in enumerate(trainloader):
 #       image = image.type(torch.DoubleTensor)
      yhat = model(image)
      loss = criterion(yhat, y)
      optimizer.zero_grad()
#      loss.backward(retain_graph = True)
      loss.backward()
      optimizer.step()
      if(idx % 100 == 0):
        print("Train: {} [loss: {}]".format(idx, loss)) 
#    loss = loss/len(trainloader)
  loss = 0
  correct = 0
  model.eval()
  for _, (data, y) in enumerate(validloader):
      output = model(data)
      loss += criterion(output, y).data.item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += (y == torch.tensor(np.argmax(output.detach().numpy(), axis = 1))).sum().item()
        
  loss /= len(validloader.dataset)
  loss_list.append(loss)
  print("Val average_loss: {} accuracy: {} ({}/{})".format(loss, correct / len(validloader.dataset), correct, len(validloader.dataset)))
  accuracy_list.append(correct/len(validloader.dataset))
# Type your code here

epoch: 0
Train: 0 [loss: 2.4319069385528564]
Train: 100 [loss: 0.41558757424354553]
Train: 200 [loss: 0.7021271586418152]
Train: 300 [loss: 0.4790567457675934]
Train: 400 [loss: 0.08515533804893494]
Train: 500 [loss: 0.1559620499610901]
Train: 600 [loss: 0.0028314804658293724]
Train: 700 [loss: 0.013692713342607021]
Train: 800 [loss: 0.017805011942982674]
Train: 900 [loss: 0.0028968669939786196]
Val average_loss: 0.024208431236911566 accuracy: 0.69814453125 (7149/10240)
epoch: 1
Train: 0 [loss: 0.0019562947563827038]
Train: 100 [loss: 0.19248239696025848]
Train: 200 [loss: 0.3018498122692108]
Train: 300 [loss: 0.5721302628517151]
Train: 400 [loss: 0.1905655711889267]
Train: 500 [loss: 0.05315214768052101]
Train: 600 [loss: 0.0007331866072490811]
Train: 700 [loss: 0.03298536688089371]
Train: 800 [loss: 0.0015135073335841298]
Train: 900 [loss: 0.017379868775606155]
Val average_loss: 0.01887024313909933 accuracy: 0.7458984375 (7638/10240)
epoch: 2
Train: 0 [loss: 0.019550690427422523]
Tra

In [ ]:
correct/len(validloader.dataset)

0.74052734375

In [ ]:
torch.save(model.state_dict(), 'checkpoint.pth')
torch.save(model, 'model.pt')
# download checkpoint file
#files.download('model')

In [ ]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,...,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_test = test.drop(['id'], axis = 1).to_numpy()
X_test = X_test.reshape((len(test),1,28,28)).astype(np.uint8)
X_test = torch.tensor(X_test)
X_test = X_test/255

In [ ]:
model.eval()
y_test = model(torch.tensor(X_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [ ]:
pred = y_test.data.max(1, keepdim=True)[1]

In [ ]:
sample = pd.read_csv('/content/sample_submission.csv')

In [ ]:
sample['label'] = pred.detach().numpy()
sample.head()

,id,label
0,0,3
1,1,0
2,2,2
3,3,6
4,4,7


In [ ]:
sample.to_csv('b.csv',index=False)